In [1]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [2]:
from transformers import pipeline

classifier = pipeline("text-classification")

2023-05-14 12:50:24.493883: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 12:50:24.518149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 12:50:24.894364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not

- 각 파이프라인은 텍스트 문자열(또는 문자열의 리스트)을 입력으로 받고 예측 리스트를 반환함

- 각 예측은 하나의 파이썬 딕셔너리이므로 판다스의 DataFrame으로 멋지게 출력할 수 있음

In [3]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901545


* 모델은 텍스트가 부정적이라고 확신함 화난 고객의 불만을 처리하고 있다는 뜻

* 감성분석 작업에서 파이프라인은 POSITIVE와 NEGATIVE 레이블 중 하나를 반환함

  - 출력되지 않은 레이블의 점수는 1-score로 계산하면 되기 때문

### 1.5.2 개체명 인식

* NLP에서는 제품, 장소, 사람 같은 실제 객체를 **개체명(named entity)** 라고 함

  - 이런 개체명을 텍스트에서 추출하는 작업을 **개체명 인식(named entity reconition, NER)** 이라고 함
  
* 해당 파이프라인을 로드하고 고객 리뷰를 전달해 NER을 적용

In [4]:
ner_tagger = pipeline("ner", aggregation_strategy = "simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


* 이 파이프라인은 모든 개체명을 감지하고 ORG(조직), LOC(위치), PER(사람) 같은 카테고리에 할당함

  - 이 예에서 모델 예측에 따라 단어를 그룹화하기 위해 aggregation_strategy 매개변수 사용
  
* 점수는 모델이 개체명을 얼마나 확신하는지 나타냄


앞의 표에서 word 열에서 해시(hash) 기호(#)는 모델의 토크나이저가 생성함

### 1.5.3 질문 답변

* 질문 답변에서는 텍스트 구절과 함게 답을 얻고 싶은 질문을 모델에 전달하고, 모델은 답변 텍스트를 반환함

In [5]:
reader = pipeline("question-answering")
question = "what does the customer want?"
outputs = reader(question = question, context = text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.642406,335,358,an exchange of Megatron


* 파이프라인은 답변과 함께 (NER처럼) 답이 위치한 문자 인덱스에 해당하는 start와 end 정수도 반환

  - 질문 답변에는 여러 유형이 있는데, 이 내용은 7장에서 알아봄
  
* 이 예제의 경우 답변을 텍스트에서 직접 추출하기 때문에 **추출적 질문 답변(extractive question answering)** 이라고 함

### 1.5.4 요약

* 텍스트 요약(text summarization)의 목표는 긴 텍스트를 입력으로 받고 관련 사실이 모두 포함된 간단한 버전을 생성하는 것

* 모델이 논리적인 텍스트를 생성해야 하므로 이전 문제보다 훨씬 더 복잡한 작업임

In [6]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length = 90, clean_up_tokenization_spaces = True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand my dilemma.


* 요약이 아주 나쁘지는 않음. 원본 텍스트의 일부가 복사됐지만 모델이 문제의 핵심을 감지하고 범블비가 불만을 제기한 사람임을 정확히 구분함

* 이 예에서는 파이프라인에 max_length와 clean_up_tokenization_spaces 키워드 매개변수를 사용함

### 1.5.5 번역

* 요약과 마찬가지로, 번역 또한 텍스트를 생성해 출력하는 작업

* 번역 파이프라인을 사용해 영어 텍스트를 독일어로 번역

In [7]:
translator = pipeline("translation_en_to_de", 
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

/home/bread/anaconda3/envs/py31011/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


* 이번에도 매우 잘 번역함. 파이프라인의 기본 모델을 오버라이드 해서 애플리케이션에 가장 잘 맞는 모델을 선택함

### 1.5.6 텍스트 생성

* 자동 완성 기능으로 고객 피드백에 빠르게 응답하고 싶다고 가정

  - 이때 텍스트 생성 모델을 사용

In [8]:
from transformers import set_seed
set_seed(42) # 동일 결과를 재현하기 위해 지정

In [9]:
# from transformers import pipeline, set_seed
# generator = pipeline('text-generation', model='gpt2', tokenizer='gpt2', pad_token_id=0, eos_token_id=1)

In [10]:
import transformers

print(transformers.__version__)

4.28.1


In [11]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=1)

fatal: 대상 경로가('nlp-with-transformers') 이미 있고 빈 디렉터리가 아닙니다.
/home/bread/A_data_practice/transformer/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.28.1
Using datasets v2.12.0
Using accelerate v0.18.0
Using sentencepiece v0.1.98
Using sacremoses v0.0.41


In [12]:
from transformers import set_seed
from transformers import pipeline
set_seed(42) # 동일 결과를 재현하기 위해 지정

generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

/home/bread/anaconda3/envs/py31011/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. For one, I
received a paper file from a store just outside your country and sent it to you
by a customer whose name was not listed.

To remove any further records, however, I can make sure I can send a copy of my
purchase to you via USPS.

Any questions or comments?

I need


In [13]:
import sys
print(sys.version)

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]


In [14]:
import torch
print(torch.__version__)


2.0.0+cu117


* 허깅페이스 트랜스포머스 생태계에는 최신 머신러닝 워크플로 대부분을 지원하는 유용한 도구가 많음

## 1.6 허깅페이스 생태계

* 허깅페이스 트랜스포머스는 NLP와 머신러닝 프로젝트의 속도를 높이는 다양한 라이브러리와 도구를 갖춘 생태계로 빠르게 성장함


### 1.6.1 허깅페이스 허브

* 허브는 모델 가중치 외에 데이터셋과 평가 지표 계산을 위한 스크립트도 호스팅하고 있어 공개된 결과를 재현하거나 애플리케이션에 활용 가능한 추가 데이터를 얻는 데 유용함

  - 허브는 모델과 데이터셋 내용을 문서화한 모델 카드와 데이터셋 카드도 제공하므로 자신에게 적합한지 여부를 정보에 입각해 결정하는데 도움이 됨

*(NOTE) 파이토치와 텐서플로도 각각 허브를 제공하므로 허깅페이스 허브에 특정 모델이나 데이터셋이 없다면 이들 허브를 확인해보세요.* 

### 1.6.2 허깅페이스 토크나이저

  - https://github.com/huggingface/transformers/blob/main/README_ko.md

* 허깅페이스 토크나이저는 다양한 토큰화 전략을 제공, 러스트 백엔드 덕분에 매우 빠르게 텍스트를 토큰화함

* 입력을 정규화하거나 모델 출력을 적절한 포맷으로 변환하는 등의 모든 전처리, 사후처리 단계를 처리함

  - 허깅페이스 토크나이저를 사용하면, 허깅페이스 트랜스포머스로 사전 훈련된 모델 가중치를 로드하는 방식으로 토크나이저를 로드할 수 있음
  
  - 모델을 훈련하고 평가하려면 데이터셋과 평가 방법이 필요함
  
     + 이런 일을 담당하느 허깅페이스 데이터셋을 살펴봄

### 1.6.3 허깅페이스 데이터셋

  - https://github.com/huggingface/datasets
  
  - https://huggingface.co/datasets

* 허깅페이스 데이터셋은 이련의 과정을 허브에서 찾을 수 있는 수천 개의 데이터셋에 대한 표준 인터페이스를 제공해 단순화 함

  - 스마트한 캐싱을 제공하고 메모리 매핑이라는 특별한 메커니즘을 활용해 램 부족을 피함
   
    + 메모리 매핑은 파일 내용을 가상 메모리에 저장하고 여러 개의 프로세스로 더 효율적으로 파일을 수정함
    
    + 이 라이브러리는 판다스와 넘파이와 같은 인기 있는 프레임 워크와 상호 운여이 가능하며, 즐겨 사용하는 데이터 랭글링 도구를 그대로 사용해도 좋음

* 허깅페이스 트랜스포머스, 허깅페이스 토크나이저, 허깅페이스 데이터셋 라이브러리를 사용하면 자신만의 트랜스포머 모델을 훈련할 수 있음

  - 훈련 루프를 미세하게 제어할 때가 있는데, 이를 위해 이 생태계의 마지막 라이브러리인 허깅페이스 엑셀러레이트가 필요함

### 1.6.4 허깅페이스 액셀러레이트

  - https://github.com/huggingface/accelerate

* 파이토치로 훈련 스크립트를 작성해봤다면 랩톱에서 실행되는 코드를 회사 클러스터로 포팅할 때 어려웠던 경험이 있을 것임

* 허깅페이스 액셀러레이트는 사용자 정의의 로직을 처리하는 일반적인 훈련 루프에 훈련 인프라에 필요한 추상화 층을 추가함

  - 말 그대로 필요한 인프라 전환을 단순화해 워크플로우를 가속화함